In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from pytorchcv.model_provider import get_model
import torchvision.models as models

In [2]:
train_transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [3]:
cache = "/scratch/doa240/.cache/datasets"
train_data = torchvision.datasets.CIFAR10(root=cache, train=True, download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=4)

Files already downloaded and verified


In [4]:
classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

In [5]:
# model = models.resnet50()
model = get_model('resnet110_cifar10', pretrained=True, root="/scratch/doa240/.torch/models")
model.eval()

CIFARResNet(
  (features): Sequential(
    (init_block): ConvBlock(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activ): ReLU(inplace=True)
    )
    (stage1): Sequential(
      (unit1): ResUnit(
        (body): ResBlock(
          (conv1): ConvBlock(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activ): ReLU(inplace=True)
          )
          (conv2): ConvBlock(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (activ): ReLU(inplace=True)
      )
      (unit2): ResUnit(
        (body): ResBlock(
          (co

In [6]:
import torch.nn as nn
# model.classifier[3] = nn.Linear(4096,1024)
# model.classifier[6] = nn.Linear(1024,10)
# model.output.fc2.fc = nn.Linear(4096,1024)
# model.fc = nn.Linear(2048,10)
# model.eval()

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
model.to(device)

CIFARResNet(
  (features): Sequential(
    (init_block): ConvBlock(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activ): ReLU(inplace=True)
    )
    (stage1): Sequential(
      (unit1): ResUnit(
        (body): ResBlock(
          (conv1): ConvBlock(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activ): ReLU(inplace=True)
          )
          (conv2): ConvBlock(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (activ): ReLU(inplace=True)
      )
      (unit2): ResUnit(
        (body): ResBlock(
          (co

In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [10]:
import time

In [11]:
# for epoch in range(10):  # loop over the dataset multiple times

#     running_loss = 0.0
#     start_time = time.time()
#     for i, data in enumerate(trainloader, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data[0].to(device), data[1].to(device)

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         output = model(inputs)
#         loss = criterion(output, labels)
#         loss.backward()
#         optimizer.step()

#         #Time
#         end_time = time.time()
#         time_taken = end_time - start_time

#         # print statistics
#         running_loss += loss.item()
#         if (i+1) % 5000 == 0:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
#             print('Time:',time_taken)
#             running_loss = 0.0

# print('Finished Training of AlexNet')

In [12]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

test_data = torchvision.datasets.CIFAR10(root=cache, train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False, num_workers=4)

Files already downloaded and verified


In [13]:
#Testing Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 96.31 %


In [14]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %.2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of Airplane : 93.10 %
Accuracy of   Car : 100.00 %
Accuracy of  Bird : 93.94 %
Accuracy of   Cat : 88.24 %
Accuracy of  Deer : 96.30 %
Accuracy of   Dog : 96.97 %
Accuracy of  Frog : 94.44 %
Accuracy of Horse : 100.00 %
Accuracy of  Ship : 100.00 %
Accuracy of Truck : 100.00 %


In [15]:
# #Verifying average accuracy of the network
# avg = 0
# for i in range(10):
#   temp = (100 * class_correct[i] / class_total[i])
#   avg = avg + temp
# avg = avg/10
# print('Average accuracy = ', avg)  